# Neural Network Activation Function Comparison Lab

1. Data Preparation: Fashion MNIST + Split the dataset into training, validation, and test sets.
2. Model Implementation: FNN for 5 times
3. Training: Use the validation set to tune hyper-parameters and prevent over-fitting
4. Evaluation: Consider metrics like accuracy (for classification) or mean squared error (for regression) + Plot the training and validation loss curves for each model
5. Analysis

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

ModuleNotFoundError: No module named 'torch'